# Notebook to build an Airbnb dataset in the US

In [ ]:
import csv
import pandas as pd
import geopandas as gpd
from shapely.geometry import *
from langdetect import detect
import textstat
from fuzzywuzzy import fuzz

from scripts import clean_text

### Choose cities and shapefiles

In [ ]:
cities = ["san_francisco","chicago"]
tracts = ["california_tracts_2018/cb_2018_06_tract_500k.shp","illinois_tracts_2018/tl_2018_17_tract.shp"]
modif = [False,True]

### Loading listings
 - Loading the dataset
 - Loading the shapefile for US Tracts
 - Using coordinates to determine tracts
 - Concatenating the description fields
 - Doing some data cleaning

In [ ]:
frames = []
for i in range(len(cities)):
    new_listings = pd.read_csv("../original_data/airbnb_data/" + cities[i] + "_listings.csv")[['id','summary','space','description','neighborhood_overview','notes','interaction','house_rules','host_about','host_id','latitude','longitude']]
    
    gdf = gpd.read_file("../original_data/" + tracts[i]).to_crs(epsg=4326)
    tract_to_add = []
    for k in range(new_listings.shape[0]):
        point = Point(new_listings["longitude"][k], new_listings["latitude"][k])
        value = None
        for j in range(gdf.shape[0]):
            if gdf["geometry"][j].contains(point):
                if (modif[i]):
                    value = "1400000US" + gdf["GEOID"][j]
                else:
                    value = gdf["AFFGEOID"][j]
                break
        tract_to_add.append(value)
    new_listings["tract"] = tract_to_add
    
    frames.append(new_listings)
listings = pd.concat(frames).reset_index(drop=True)

columns = ['summary','space','description','neighborhood_overview','notes','interaction','house_rules','host_about']
to_add = []
for i in range(listings.shape[0]):
    description = ""
    for j in columns:
        item = listings[j][i]
        if (isinstance(item, str)):
            description += clean_text(item)
    description = description[:-1]
    to_add.append(description)
listings['full_description'] = to_add

listings = listings[['id', 'full_description', 'host_id', 'tract']]

### First round of data cleaning
 - Removing duplicate descriptions
 - Removing too short descriptions
 - Removing rows with non-English description

In [ ]:
listings = listings.drop_duplicates('full_description').reset_index(drop=True)

In [ ]:
to_add = []
for i in range(listings.shape[0]):
    sent_count = textstat.sentence_count(listings['full_description'][i])
    to_add.append(sent_count)
listings['sent_count'] = to_add
listings = listings[listings['sent_count'] > 2].reset_index(drop=True).drop('sent_count', axis=1)

In [ ]:
to_add = []
for i in range(listings.shape[0]):
    try:
        blob = detect(listings['full_description'][i])
        to_add.append(blob)
    except:
        to_add.append("None")
listings['lang'] = to_add
listings = listings[listings['lang'] == "en"].reset_index(drop=True).drop('lang', axis=1)

### Final cleaning of the dataset
 - Removing 'None' wards (not found, in the water, etc)
 - Removing 'Fuzzy' duplicate descriptions

In [ ]:
listings = listings[listings['tract'].notnull()].reset_index(drop=True)

In [ ]:
idx = 0
while (idx < listings.shape[0]):
    print("{}/{}".format(idx, listings.shape[0]), end='\r')
    subset = listings[(listings['tract'] == listings['tract'][idx]) & (listings['host_id'] == listings['host_id'][idx])].drop(idx)
    to_remove = []
    for index, row in subset.iterrows():
        if (fuzz.ratio(listings['full_description'][idx], row['full_description']) > 60):
            to_remove.append(index)
    listings = listings.drop(listings.index[to_remove]).reset_index(drop=True)
    idx += 1

### Saving the dataset to .csv

In [ ]:
listings_out = listings[['id','full_description','tract']]

In [ ]:
listings_out.to_csv("../data/airbnb_listings_description/us_listings_description_tract.csv", index=False)